Walk through the available list of calibration images, and extract rvec/tvec info from each one.

Write results to a file.

Intended to be used to for building the ArucoBall Graph.

In [43]:
import os
import cv2
from aruco_classes import get_markers_in_single_frame

In [44]:
CALIB_IMAGES_DIR = 'calib_images/aruco_ball'
OUTPUT_FILE_PATH = 'calib_images/aruco_ball/discovered_markers.txt'

f = open(OUTPUT_FILE_PATH, 'w+')

id_rvec_tvecs_list = []

In [45]:
# for each image file in this directory:
image_files = [f for f in os.listdir(CALIB_IMAGES_DIR) if os.path.isfile(os.path.join(CALIB_IMAGES_DIR, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# output file
for image_file in image_files:
    # Process each image frame:
    print(f'Processing {image_file}')
    im = cv2.imread(os.path.join(CALIB_IMAGES_DIR, image_file))

    # extract rvecs and tvecs for markers
    id_list, rvec_list, tvec_list = get_markers_in_single_frame(im) 
    id_list = [id[0] for id in id_list]

    # format rvecs and tvecs for file writing
    rvec_list = [rvec[0] for rvec in rvec_list]
    rvec_list = [','.join(map(str,map(lambda x:str(x[0]), rvec))) for rvec in rvec_list]
    rvec_list = [list(map(float,eval(x)))for x in rvec_list]
    
    tvec_list = [rvec[0] for rvec in tvec_list]
    tvec_list = [','.join(map(str,map(lambda x:str(x[0]), rvec))) for rvec in tvec_list]
    tvec_list = [list(map(float,eval(x)))for x in tvec_list]
    tvec_list = [tvec[0] for tvec in tvec_list]
    
    id_rvec_tvecs_list.append([id_list, rvec_list, tvec_list])
    
    # write line to file
    line = ','.join(f'[{id},{rvec},{tvec}]' for id, rvec, tvec in zip(id_list, rvec_list, tvec_list))
    f.write(line+'\n')

Processing IMG_6129.png
Processing IMG_6138.png
Processing IMG_6139.png
Processing IMG_6143.png
Processing IMG_6142.png
Processing IMG_6140.png
Processing IMG_6141.png
Processing IMG_6144.png
Processing IMG_6146.png
Processing IMG_6134.png
Processing IMG_6135.png
Processing IMG_6136.png
Processing IMG_6132.png
Processing IMG_6133.png
Processing IMG_6131.png
Processing IMG_6130.png


In [46]:
id_rvec_tvecs_list[0]

[[23, 18, 6, 36, 16, 7],
 [[3.3848840167697816, 1.203209884143662, 1.0615563218691246],
  [0.7115784024185193, -21.13168908092291, -6.667376381056031],
  [0.8576076890276597, -0.1771549039986433, -0.5940948744123354],
  [1.6052130494930148, -1.0185215792022366, 0.7614069406452807],
  [1.4186286503055203, -0.9465607522552282, 0.31195190490892194],
  [39.84755134523583, -7.249307756047214, 160.24370327564864]],
 [0.06753417460268687,
  0.010125841167357453,
  0.1469396522073032,
  0.08641918047572945,
  0.030070567560213608,
  3.178312522840132]]

In [47]:
for id, rvecs, tvecs in id_rvec_tvecs_list:
    print(id)
    print(rvecs)
    print(tvecs)
    print()

[23, 18, 6, 36, 16, 7]
[[3.3848840167697816, 1.203209884143662, 1.0615563218691246], [0.7115784024185193, -21.13168908092291, -6.667376381056031], [0.8576076890276597, -0.1771549039986433, -0.5940948744123354], [1.6052130494930148, -1.0185215792022366, 0.7614069406452807], [1.4186286503055203, -0.9465607522552282, 0.31195190490892194], [39.84755134523583, -7.249307756047214, 160.24370327564864]]
[0.06753417460268687, 0.010125841167357453, 0.1469396522073032, 0.08641918047572945, 0.030070567560213608, 3.178312522840132]

[18, 23, 16, 4, 27, 13]
[[1.095403185707708, -2.6967469528937973, -0.13435986691108193], [-1.7846114892163127, -1.9395565591407462, -1.0255825571455723], [2.0311744324568375, -0.22371378911658826, 0.07283107736180443], [1.4092428238726413, -1.1304789279697747, 0.5554930418634468], [0.518552231032072, 1.942932916935669, -0.11494520377703722], [1.2829225566979174, 1.1576951661526043, -0.9407035965661308]]
[0.02301466980590989, 0.054209775461254384, 0.07111646497821704, 0.